## Train and Initialize My Model
_Note:  Training isn't needed here.  Learned variables are passed in for demo purposes._


In [ ]:
def initialize_model(cat_mean, 
                     cat_stdv, 
                     dog_mean, 
                     dog_stdv):

    model = {'cat_mean':cat_mean,
             'cat_stdv':cat_stdv,
             'dog_mean':dog_mean,
             'dog_stdv':dog_stdv}

    return model            

In [ ]:
model = initialize_model(0.1, 0.20, 0.3, 0.40)
print(model)

In [ ]:
def transform_request(request):
    import json        
    request_str = request.decode('utf-8')
    request_str = request_str.strip().replace('\n', ',')
    request_dict = json.loads(request_str)
    return request_dict

In [ ]:
def transform_response(response):
    import json

    response_json = json.dumps(response)
    return response_json

In [ ]:
def predict(model, request):

    cat_affinity_score = sum([ d['weight'] * d['user_score'] for d in request if 'cat' in d['tags'] ])
    dog_affinity_score = sum([ d['weight'] * d['user_score'] for d in request if 'dog' in d['tags'] ])

    # create normalized z score for compare/classify
    cat_zscore = (cat_affinity_score - model['cat_mean'])/model['cat_stdv']
    dog_zscore = (dog_affinity_score - model['dog_mean'])/model['dog_stdv']

    # classify
    if abs(cat_zscore) > abs(dog_zscore):
        if cat_zscore >= 0:
            category = "cat_lover"
        else:
            category = "cat_hater"
    else:
        if dog_zscore >= 0:
            category = "dog_lover"
        else:
            category = "dog_hater"

    response = {
        'category': category,
        'cat_affinity_score': cat_affinity_score,
        'dog_affinity_score': dog_affinity_score,
        'cat_zscore': cat_zscore,
        'cat_zscore': dog_zscore
    }

    return response

In [ ]:
class PioModel(object):
    def __init__(self,
                 transform_request,
                 transform_response,
                 model,
                 predict):
        self.transform_request = transform_request
        self.transform_response = transform_response
        self.model = model
        self.predict = predict

In [ ]:
pio_model = PioModel(transform_request,
                     transform_response,
                     model,
                     predict)

## Save/Pickle Model as `pio_model.pkl`

In [ ]:
import dill as pickle

pio_model_pkl_path = 'pio_model.pkl'

with open(pio_model_pkl_path, 'wb') as fh:
    pickle.dump(pio_model, fh)

In [ ]:
%%bash

ls -l pio_model.pkl